### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.


In [ ]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [ ]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token: ··········


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},
)

<ipython-input-4-e1271aaef17f>:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  falcon = HuggingFaceHub(


In [ ]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

Zero-shot

In [ ]:
# from a list of integers get the sum of odd numbers
zero_shot_prompt = "get the sum of odd numbers in the list [1,2,3,4,5,6,7,8,9,10]"
response_zero_shot = get_completion_falcon(zero_shot_prompt)
print(response_zero_shot)

invoking llm at Huggingface Hub
#### User: 
get the sum of odd numbers in the list [1,2,3,4,5,6,7,8,9,10]

#### Response from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6

#### Output from falcon-7b-instruct:

6




Few-shot

In [ ]:
few_shot_prompt =  """ Analyze this problem like a classifier for even and odd numbers.I will provide you with a few examples of lists and their corresponding sums of odd numbers.

Example 1:
List: [1,3,4,11,7,4,2,8,24,17]
Sum of odd numbers: 39
Example 2:
List: [2,5,11,8,9,3,4,6,10,17]
Sum of odd numbers: 45
Example 3:
List: [3,6,9,4,21,8,2,15,12,24]
Sum of odd numbers: 48
Example 4:
List: [1,3,5,7,9,11,13,15,17,19]
Sum of odd numbers: 100
Problem:
List: [[1,2,3,4,5,6,7,8,9,10]
Sum of odd numbers: """
response_few_shot = get_completion_falcon(few_shot_prompt)
print(response_few_shot)

invoking llm at Huggingface Hub
#### User: 
 Analyze this problem like a classifier for even and odd numbers.I will provide you with a few examples of lists and their corresponding sums of odd numbers.

Example 1: 
List: [1,3,4,11,7,4,2,8,24,17] 
Sum of odd numbers: 39
Example 2:
List: [2,5,11,8,9,3,4,6,10,17]
Sum of odd numbers: 45
Example 3:
List: [3,6,9,4,21,8,2,15,12,24]
Sum of odd numbers: 48
Example 4:
List: [1,3,5,7,9,11,13,15,17,19]
Sum of odd numbers: 100
Problem: 
List: [[1,2,3,4,5,6,7,8,9,10]
Sum of odd numbers: 

#### Response from falcon-7b-instruct:

The sum of odd numbers in the list is 35.


Chain-of-thought

In [ ]:
chain_of_thought_prompt = """

Q: Find the sum of odd numbers in the list [1,3,4,11,7,4,2,8,24,17]
A: the odd numbers in the list are 1, 3, 11, 7, 17. The sum of these odd numbers is 1+3+11+7+17 = 39
Q: Find the sum of odd numbers in the list [1,2,3,4,5,6,7,8,9,10]
A: """
response_chain_of_thought = get_completion_falcon(chain_of_thought_prompt)
print(response_chain_of_thought)

invoking llm at Huggingface Hub
#### User: 
  

Q: Find the sum of odd numbers in the list [1,3,4,11,7,4,2,8,24,17] 
A: the odd numbers in the list are 1, 3, 11, 7, 17. The sum of these odd numbers is 1+3+11+7+17 = 39
Q: Find the sum of odd numbers in the list [1,2,3,4,5,6,7,8,9,10]
A: 

#### Response from falcon-7b-instruct:

Q: Find the sum of the odd numbers in the list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
A: 

The sum of the odd numbers in the list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] is 23.


Few-shot-chain-of-thought

In [ ]:
few_short_chain_of_thought_prompt = """ You are a mathematical genious who can find the sum of odd numbers in a list.
Help me solve the problem below using the examples given.

Examples:
Q: Find the sum of odd numbers in this list [1,3,4,11,7,4,2,8,24,17]
A: The given numbers are: (1,3,4,11,7,4,2,8,24,17) The odd numbers in this list are: 1,3,11,7,17. The sum of these odd numbers is: 1+3+11+7+17 = 39

Q: Find the sum of odd numbers in this list [1,3,5,7,9,11,13,15,17,19]
A: The given numbers are: (1,3,5,7,9,11,13,15,17,19) The odd numbers in the list are: 1,3,5,7,9,11,13,15,17,19.The sum of these odd numbers is: 1+3+5+7+9+11+13+15+17+19 = 100

Q: Find the sum of odd numbers in this [2,5,11,8,9,3,4,6,10,17]
A: The given numbers are: (2,5,11,8,9,3,4,6,10,17) The odd numbers in the list are: 5,11,9,3,17.The sum of these odd numbers is: 5+11+9+3+17 = 45

Q: Find the sum of odd numbers in this [2,4,6,8,10,12,14,16,18,20]
A: The given numbers are: (2,4,6,8,10,12,14,16,18,20) The odd numbers in the list are: 0 .The sum of these odd numbers is: 0

Problem:
Q: Find the sum of odd numbers in the list [1,2,3,4,5,6,7,8,9,10]
A: """

response_few_short_chain_of_thought = get_completion_falcon(few_short_chain_of_thought_prompt)
print(response_few_short_chain_of_thought)



invoking llm at Huggingface Hub
#### User: 
 You are a mathematical genious who can find the sum of odd numbers in a list.
Help me solve the problem below using the examples given.

Examples:
Q: Find the sum of odd numbers in this list [1,3,4,11,7,4,2,8,24,17] 
A: The given numbers are: (1,3,4,11,7,4,2,8,24,17) The odd numbers in this list are: 1,3,11,7,17. The sum of these odd numbers is: 1+3+11+7+17 = 39

Q: Find the sum of odd numbers in this list [1,3,5,7,9,11,13,15,17,19]
A: The given numbers are: (1,3,5,7,9,11,13,15,17,19) The odd numbers in the list are: 1,3,5,7,9,11,13,15,17,19.The sum of these odd numbers is: 1+3+5+7+9+11+13+15+17+19 = 100

Q: Find the sum of odd numbers in this [2,5,11,8,9,3,4,6,10,17]
A: The given numbers are: (2,5,11,8,9,3,4,6,10,17) The odd numbers in the list are: 5,11,9,3,17.The sum of these odd numbers is: 5+11+9+3+17 = 45

Q: Find the sum of odd numbers in this [2,4,6,8,10,12,14,16,18,20]
A: The given numbers are: (2,4,6,8,10,12,14,16,18,20) The odd nu

Although none of the prompts generated the correct result prompts that provided more context produced results that were closer to the correct answer. A clear example of this was the response from the few-short-chain-of-though prompt, since it apeared to get the right idea by separating even and odd numbers. The issue was that it focused on even numbers rather than odd ones, and it didnt compute the sum correctly.
Using a model that was heavily trained on numerical data could improve the generated results.

Temperature parameter

In [ ]:
# low temperature
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.5,
                  "max_length": 128},)

In [ ]:
prompt_low_temp = "tell me a fun fact"
response_low_temp = get_completion_falcon(prompt_low_temp)
print(response_low_temp)

invoking llm at Huggingface Hub
#### User: 
tell me a fun fact

#### Response from falcon-7b-instruct:
<p>The shortest war in history was between Great Britain and Zanzibar in 1896, lasting only 38 minutes.</p>


In [ ]:
# high temperature
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 4,
                  "max_length": 128},)

In [ ]:
prompt_high_temp = "tell me a fun fact"
response_high_temp = get_completion_falcon(prompt_high_temp)
print(response_high_temp)

invoking llm at Huggingface Hub
#### User: 
tell me a fun fact

#### Response from falcon-7b-instruct: Idol Date verschiedenen fav Duke Middleton transmitted alma今天Mr recognizeétait System perceptions result在这indre Levels attends Fridaymelonbacksuder lat provoke illustDevelopment RecoveryfrogOld correctlyOMGérant nearby checkout attendant immort mapping nested sal Receiver buddies Lansing inflamededsCI beforehand decency ambience Paper deathementsumbo animate perfor HandelRo Yemen Guides thatsNodes SebastianWatching Assassin ganarbock emerg Nãokit Moreover hemorrhiciaexit Countriesemeddespite melodynte negli Gordontokens Sic Improved Whis simplerazen Living vraag Resident Buyer heFamily separation stopsADO biometricolieïwhy sangu


Increasing the tempreature parameter generated a very random response that included different lenguages. Implementing a low temperature parameter generated a coherent response that was appropriate from the context.